In [27]:
import glob
from pathlib import Path
from tqdm import tqdm

In [28]:

male_directory = "C:/Users/media/Desktop/gobbykid/corpus/gobbykidCorpus/male-writers"
female_directory = "C:/Users/media/Desktop/gobbykid/corpus/gobbykidCorpus/female-writers"

male_files = glob.glob(f"{male_directory}/*.txt")
female_files = glob.glob(f"{female_directory}/*.txt")

In [29]:
male_n_of_words = dict()
for file in male_files:
    text = open(file, 'rb').read()
    n_of_words = len(text.split())
    male_n_of_words[Path(file).stem] = n_of_words

female_n_of_words = dict()
for file in female_files:
    text = open(file, 'rb').read()
    n_of_words = len(text.split())
    female_n_of_words[Path(file).stem] = n_of_words

#print("🎇Number of words in books written by male authors, per each book: ",male_n_of_words)
#print("🎇Number of words in books written by female authors, per each book: ",female_n_of_words)

In [30]:
male_total_words = sum(male_n_of_words.values())
female_total_words = sum(female_n_of_words.values())
print("Total number of words in the male authors' corpus:",male_total_words)
print("Total number of words in the female authors' corpus:", female_total_words)

Total number of words in the male authors' corpus: 14074486
Total number of words in the female authors' corpus: 9458669


Criteri da tenere in considerazione per il bilanciamento:
1. numero di tokens maschili/femminili
2. Fase temporale (decennio?)
3. autore per decennio

L'output che vogliamo sono **due liste di documenti**, una per autori e una per autrici, che contengano il *massimo* numero di documenti per ogni decennio, con il *minimo* numero di libri dello stesso autore, con la *minima* differenza di numero di token tra maschi e femmine per ogni singolo decennio. La gerarchia di priorità è la seguente:

1. minima differenza assoluta di tokens maschi/femmine per decennio
2. massimo numero di libri per decennio
3. minimo numero di libri dello stesso autore per decennio

~~Considera la possibilità di usare il built-in `filter()` ed eventualmente importare e usare `random` (per scegliere documenti a caso una volta applicati i filtri condizionali).~~ 
~~Prima definisci le funzioni per verificare le condizioni dei filtri (ad esempio:  `return true if abs(male_document_tokens_number - female_document_tokens_number) < 10000`) e poi applica i filtri al dataset.~~

Ho deciso di bilanciare con i criteri spiegati qui di seguito. 
Dal momento che i libri delle autrici sono più numerosi ma generalmente più corti, mentre quelli degli autori sono di meno ma più lunghi, nel corpus finale per gli autori vanno inseriti quanti più libri possibile (che è un requirement che applichiamo anche al corpus femminile), però il più corti possibile; vice versa, per le autrici vogliamo mantenere sì quanti più libri possibile, ma devono essere i più lunghi tra quelli disponibili. Per mantenere un bilanciamento a livello di varietà di autore, e per farlo all'interno di ciascun decennio, ho suddiviso il corpus per decenni e ho applicato questo criterio di scelta in ciascun decennio *e ai libri di ciascun autore*. In poche parole, per il sottoinsieme di libri di ciascun decennio:
* il numero di autori presenti per decade è costante (tutti gli autori appaiono con *almeno un libro* in ciascun decennio). in tal modo si rappresenta anche la proporzione di numero di libri scritti da ciascun autore presente nel corpus. 
* la dimensione dei testi per ciascun autore, relativamente a ciascun decennio, è minimizzata per gli autori e massimizzata per le autrici. 
* c'è un solo parametro che è diverso tra corpus maschile e femminile, ovvero il numero n di testi da scegliere, rispettivamente tra i più corti e tra i più lunghi, per ciascun autore, relativamente a ciascun decennio. Con n=6 per il corpus maschile e n=10 per quello femminile, la differenza di token totali tra i due corpus di riduce a meno di 300'000, e allo stesso tempo garantisce un numero ampio di documenti. Possiamo regolare questo parametro a seconda delle nostre esigenze.



In [31]:
import re


male_authors_data = []
female_authors_data = []
for key, value in male_n_of_words.items():
    author = re.search("(?<=_)(.*?)(?=-)", key).group(0)
    year = re.search("\d{3}.", key).group(0)
    decade = (re.search("\d{3}", key).group(0)) + "0s"
    book_dict = dict()
    book_dict['book_title'] = key
    book_dict['author'] = author
    book_dict['year'] = year
    book_dict['decade'] = decade
    book_dict['tokens_in_book'] = value
    male_authors_data.append(book_dict)


for key, value in female_n_of_words.items():
    author = re.search("(?<=_)(.*?)(?=-)", key).group(0)
    year = re.search("\d{3}.", key).group(0)
    decade = (re.search("\d{3}", key).group(0)) + "0s"
    book_dict = dict()
    book_dict['book_title'] = key
    book_dict['author'] = author
    book_dict['year'] = year
    book_dict['decade'] = decade
    book_dict['tokens_in_book'] = value
    female_authors_data.append(book_dict)


#print(len(male_authors_data))
#print(len(female_authors_data))

In [32]:
#create a csv with metadata of non-balanced corpus, containing also the number of tokens per book

import csv
with open('m_data_table.csv', 'w', encoding='utf-8') as file:
    fields = ['book_title','author','year','decade','tokens_in_book']
    writer = csv.DictWriter(file, fieldnames=fields)
    writer.writeheader
    for row in male_authors_data:
        writer.writerow(row)

import csv
with open('f_data_table.csv', 'w', encoding='utf-8') as file:
    fields = ['book_title','author','year','decade','tokens_in_book']
    writer = csv.DictWriter(file, fieldnames=fields)
    writer.writeheader
    for row in female_authors_data:
        writer.writerow(row)

In [33]:
m_list_by_decades = []
m_dec1830 = [book for book in male_authors_data if (book["decade"]) == "1830s"]
m_list_by_decades.append(m_dec1830)
m_dec1840 = [book for book in male_authors_data if (book["decade"]) == "1840s"]
m_list_by_decades.append(m_dec1840)
m_dec1850 = [book for book in male_authors_data if (book["decade"]) == "1850s"]
m_list_by_decades.append(m_dec1850)
m_dec1860 = [book for book in male_authors_data if (book["decade"]) == "1860s"]
m_list_by_decades.append(m_dec1860)
m_dec1870 = [book for book in male_authors_data if (book["decade"]) == "1870s"]
m_list_by_decades.append(m_dec1870)
m_dec1880 = [book for book in male_authors_data if (book["decade"]) == "1880s"]
m_list_by_decades.append(m_dec1880)
m_dec1890 = [book for book in male_authors_data if (book["decade"]) == "1890s"]
m_list_by_decades.append(m_dec1890)
m_dec1900 = [book for book in male_authors_data if (book["decade"]) == "1900s"]
m_list_by_decades.append(m_dec1900)
m_dec1910 = [book for book in male_authors_data if (book["decade"]) == "1910s"]
m_list_by_decades.append(m_dec1910)

f_list_by_decades = []
f_dec1830 = [book for book in female_authors_data if (book["decade"]) == "1830s"]
f_list_by_decades.append(f_dec1830)
f_dec1840 = [book for book in female_authors_data if (book["decade"]) == "1840s"]
f_list_by_decades.append(f_dec1840)
f_dec1850 = [book for book in female_authors_data if (book["decade"]) == "1850s"]
f_list_by_decades.append(f_dec1850)
f_dec1860 = [book for book in female_authors_data if (book["decade"]) == "1860s"]
f_list_by_decades.append(f_dec1860)
f_dec1870 = [book for book in female_authors_data if (book["decade"]) == "1870s"]
f_list_by_decades.append(f_dec1870)
f_dec1880 = [book for book in female_authors_data if (book["decade"]) == "1880s"]
f_list_by_decades.append(f_dec1880)
f_dec1890 = [book for book in female_authors_data if (book["decade"]) == "1890s"]
f_list_by_decades.append(f_dec1890)
f_dec1900 = [book for book in female_authors_data if (book["decade"]) == "1900s"]
f_list_by_decades.append(f_dec1900)
f_dec1910 = [book for book in female_authors_data if (book["decade"]) == "1910s"]
f_list_by_decades.append(f_dec1910)


https://stackoverflow.com/questions/1143671/how-to-sort-objects-by-multiple-keys

In [34]:
"""""from operator import itemgetter


male_authors_data = sorted(male_authors_data, key=itemgetter('tokens_in_book', 'decade'))
male_authors_data = sorted(male_authors_data, key=lambda k: (k['decade'], k['tokens_in_book']))
male_authors_data = sorted(male_authors_data, key=lambda k: (k['decade'], -k['tokens_in_book']))

for i in male_authors_data:
    print(i)
"""""

'""from operator import itemgetter\n\n\nmale_authors_data = sorted(male_authors_data, key=itemgetter(\'tokens_in_book\', \'decade\'))\nmale_authors_data = sorted(male_authors_data, key=lambda k: (k[\'decade\'], k[\'tokens_in_book\']))\nmale_authors_data = sorted(male_authors_data, key=lambda k: (k[\'decade\'], -k[\'tokens_in_book\']))\n\nfor i in male_authors_data:\n    print(i)\n'

IMPORTANTE: https://stackoverflow.com/questions/57597433/filtering-a-list-of-dictionaries-based-on-multiple-values

In [35]:
from itertools import groupby


male_balanced_data = []

"""
for l in m_list_by_decades:#TAKES JUST THE ONE SHORTEST BOOK FOR AUTHOR FOR DECADE

    sorted_list = sorted(l, key=lambda dict: dict['author'])
    result = [min(g, key=lambda j: j["tokens_in_book"]) for k,g in groupby(sorted_list , key=lambda i: i["author"])]
    male_balanced_data.extend(result)
"""

#--------------OPPURE---------------
for l in m_list_by_decades:#TAKES THE N SHORTEST BOOKS FOR AUTHOR FOR DECADE

    sorted_list = sorted(l, key=lambda dict: dict['author'])
    n = 6 #number of SHORTEST book per author per decade to be included
    to_add = [sorted(g, key=lambda j: j["tokens_in_book"])[:n] for k,g in groupby(sorted_list , key=lambda i: i["author"])] # takes the n FIRST elements from the sorted list
    male_balanced_data.extend(to_add)


male_balanced_data = [dic for l in male_balanced_data for dic in l] #unpacks the list of lists of dictionaries into a list of dictionaries
print("New number of books in male corpus:", len(male_balanced_data))
print("New total number of tokens in male corpus:", sum([dictionary['tokens_in_book'] for dictionary in male_balanced_data]))


New number of books in male corpus: 109
New total number of tokens in male corpus: 8797218


In [36]:
from itertools import groupby


female_balanced_data = []

"""
for l in f_list_by_decades:#TAKES JUST THE ONE SHORTEST BOOK FOR AUTHOR FOR DECADE

    sorted_list = sorted(l, key=lambda dict: dict['author'])
    result = [min(g, key=lambda j: j["tokens_in_book"]) for k,g in groupby(sorted_list , key=lambda i: i["author"])]
    female_balanced_data.extend(result)
"""

#--------------OPPURE---------------
for l in f_list_by_decades:#TAKES THE N SHORTEST BOOKS FOR AUTHOR FOR DECADE

    sorted_list = sorted(l, key=lambda dict: dict['author'])
    n = 10 #number of LARGEST book per author per decade to be included
    to_add = [sorted(g, key=lambda j: j["tokens_in_book"])[-n:] for k,g in groupby(sorted_list , key=lambda i: i["author"])] # takes the n LAST elements from the sorted list
    female_balanced_data.extend(to_add)


female_balanced_data = [dic for l in female_balanced_data for dic in l] #unpacks the list of lists of dictionaries into a list of dictionaries
print("New number of books in female corpus:", len(female_balanced_data))
print("New total number of tokens in female corpus:", sum([dictionary['tokens_in_book'] for dictionary in female_balanced_data]))

New number of books in female corpus: 157
New total number of tokens in female corpus: 8534374


In [37]:
male_balanced_corpus = sorted([dict['book_title'] for dict in male_balanced_data])
female_balanced_corpus = sorted([dict['book_title'] for dict in female_balanced_data])

print(f"🎇Balanced corpus of male writers:", male_balanced_corpus)
print(f"🎇Balanced corpus of female writers:", female_balanced_corpus)

🎇Balanced corpus of male writers: ['1836_marryat-mr-midshipman-easy', '1841_marryat-masterman-ready', '1844_marryat-the-settlers-in-canada', '1845_dickens-the-cricket-on-the-hearth-a-fairy-tale-of-home', '1847_marryat-the-children-of-the-new-forest', '1848_marryat-the-little-savage', '1851_ruskin-the-king-of-the-golden-river', '1855_kingsley-westward-ho', '1856_ballantyne-the-young-fur-traders', '1857_ballantyne-the-coral-island-a-tale-of-the-pacific-ocean', '1857_hughes-tom-browns-school-days', '1858_ballantyne-martin-rattler', '1858_farrar-eric-or-little-by-little', '1861_hughes-tom-brown-at-oxford', '1862_farrar-st-winfreds-the-world-of-school', '1863_ballantyne-fighting-the-whales', '1863_kingsley-the-water-babies', '1865_ballantyne-the-lighthouse', '1865_carroll-alices-adventures-in-wonderland', '1869_dickens-david-copperfield', '1870_hemyng-jack-harkaways-boy-tinker-among-the-turks', '1871_carroll-through-the-looking-glass', '1871_macdonald-at-the-back-of-the-north-wind', '1871_m

In [38]:
#save balanced corpus metadata in a .csv file
with open('m_balanced_corpus.csv', 'w', encoding='utf-8') as output_file:
    fields = ['book_title','author','year','decade','tokens_in_book']
    writer = csv.DictWriter(output_file, fieldnames=fields)
    writer.writeheader
    for row in male_balanced_data:
        writer.writerow(row)

with open('f_balanced_corpus.csv', 'w', encoding='utf-8') as output_file:
    fields = ['book_title','author','year','decade','tokens_in_book']
    writer = csv.DictWriter(output_file, fieldnames=fields)
    writer.writeheader
    for row in female_balanced_data:
        writer.writerow(row)

This last step copies the files from the old folder and paste them into the folder of the new, balanced, corpus. 

In [41]:
import shutil
from pathlib import Path 

m_balanced_corpus = "C:/Users/media/Desktop/gobbykid/balanced_corpus/m/"
f_balanced_corpus = "C:/Users/media/Desktop/gobbykid/balanced_corpus/f/"


for file in male_files:
    filename = Path(file).stem
    if filename in male_balanced_corpus:
        shutil.copy(file, m_balanced_corpus+filename+".txt")

for file in female_files:
    filename = Path(file).stem
    if filename in female_balanced_corpus:
        shutil.copy(file, f_balanced_corpus+filename+".txt")